In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors

import flopy

In [ ]:
stage_cmap = colors.ListedColormap(["cyan"])

In [ ]:
sim = flopy.mf6.MFSimulation.load(sim_ws="model_ss")
gwf = sim.get_model()
top = gwf.dis.top.array

#### create structured grid for dflowfm

In [ ]:
nrow, ncol = gwf.modelgrid.nrow, gwf.modelgrid.ncol
dflowfm_modelgrid = flopy.discretization.StructuredGrid(
    nlay=1,
    nrow=nrow,
    ncol=ncol,
    delr=gwf.modelgrid.delr,
    delc=gwf.modelgrid.delc,
    xoff=gwf.modelgrid.xoffset,
    yoff=gwf.modelgrid.yoffset,
    botm=top.copy().reshape(1, nrow, ncol),
    top=np.full((nrow, ncol), 5., dtype=float),
)

In [ ]:
dflowfm_stage = gwf.ghb.stress_period_data.array[0]["bhead"][0]
dflowfm_stage = np.full((nrow, ncol), dflowfm_stage, dtype=float)

#### process MODFLOW output

In [ ]:
cbc = gwf.output.budget()

In [ ]:
cbc.get_unique_record_names()

In [ ]:
sat = cbc.get_data(text="DATA-SAT")[0]

In [ ]:
sat.dtype.names

In [ ]:
sat["sat"].shape, sat["sat"].min(), sat["sat"].max()

In [ ]:
spdis = cbc.get_data(text="DATA-SPDIS")[0]
spdis.dtype.names

In [ ]:
qx, qy, qz = flopy.utils.postprocessing.get_specific_discharge(spdis, gwf)

#### plot results

In [ ]:
mv = flopy.plot.PlotMapView(model=gwf)
ps = mv.plot_array(sat["sat"])
plt.colorbar(ps)

In [ ]:
head = gwf.output.head().get_data()

In [ ]:
head.max()

In [ ]:
dtw = gwf.dis.top.array - head[0]

In [ ]:
mv = flopy.plot.PlotMapView(model=gwf)
pv = mv.plot_array(dtw)
plt.colorbar(pv)

In [ ]:
extent = (0, 11, -20, 5)
fig, ax = plt.subplots(nrows=1, ncols=1)
fig.set_figheight(4)
fig.set_figwidth(5)

xs = flopy.plot.PlotCrossSection(model=gwf, line={"row": 5}, extent=extent)
xs.ax.axhline(y=0.0, lw=0.5, ls=":", color="black")
pv = xs.plot_array(head, head=head)
xs.plot_vector(qx, qy, qz, head=head, normalize=True)
xs.plot_grid()
plt.colorbar(pv);

xs_df = flopy.plot.PlotCrossSection(modelgrid=dflowfm_modelgrid, ax=ax, line={"row": 5}, extent=extent)
ps = xs_df.plot_array(dflowfm_stage, head=dflowfm_stage, cmap=stage_cmap);